In [3]:
#! pip install kagglehub
! pip install matplotlib

In [4]:
from torch.utils.data import DataLoader, Dataset
import torch
import torch.nn as nn
import torch.optim as optim

import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

import matplotlib.pyplot as plt

In [5]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("zalando-research/fashionmnist")

print("Path to dataset files:", path)

/usr/local/python/3.12.1/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Path to dataset files: /home/codespace/.cache/kagglehub/datasets/zalando-research/fashionmnist/versions/4


In [6]:
df = pd.read_csv(path + "/fashion-mnist_train.csv")

df.head()

,label,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783,pixel784
0,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,6,0,0,0,0,0,0,0,5,0,...,0,0,0,30,43,0,0,0,0,0
3,0,0,0,0,1,2,0,0,0,0,...,3,0,0,0,0,1,0,0,0,0
4,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [7]:
sample_df = df.sample(n=10000, random_state=42)

In [8]:
sample_df.shape

(10000, 785)

In [9]:
X = sample_df.iloc[:, 1:].values
y = sample_df.iloc[:, :1].values


In [10]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


# Pre - Processing

In [11]:
st_scaler = StandardScaler()

In [12]:

X_train = st_scaler.fit_transform(X_train)
X_test = st_scaler.transform(X_test)

In [13]:
X_train

array([[-0.0135594 , -0.03425274, -0.04112689, ..., -0.15808801,
        -0.08864994, -0.04598137],
       [-0.0135594 , -0.03425274, -0.04112689, ..., -0.15808801,
        -0.08864994, -0.04598137],
       [-0.0135594 , -0.03425274, -0.04112689, ..., -0.15808801,
        -0.08864994, -0.04598137],
       ...,
       [-0.0135594 , -0.03425274, -0.04112689, ..., -0.15808801,
        -0.08864994, -0.04598137],
       [-0.0135594 , -0.03425274, -0.04112689, ..., -0.15808801,
        -0.08864994, -0.04598137],
       [-0.0135594 , -0.03425274, -0.04112689, ..., -0.15808801,
        -0.08864994, -0.04598137]], shape=(8000, 784))

# Dataset Class

In [14]:
# create custom dataset class

class FashionMNISTDataset(Dataset):

    def __init__(self, features, labels):
        print(type(features))
        print(type(labels))
        self.features = torch.tensor(features, dtype=torch.float32)
        self.labels = torch.tensor(labels, dtype=torch.long)


    def __len__(self):
        return len(self.features)


    def __getitem__(self, idx):
        return self.features[idx], self.labels[idx]

In [15]:
train_dataset = FashionMNISTDataset(X_train, y_train)
test_dataset = FashionMNISTDataset(X_test, y_test)

<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


In [16]:
train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

# NN Model 

In [19]:
class MyNN(nn.Module):

    def __init__(self, num_features):

        super().__init__()

        self.model = nn.Sequential(

            nn.Linear(num_features, 128),
            nn.ReLU(),
            nn.Linear(128, 64),
            nn.ReLU(),
            nn.Linear(64, 16),
            # nn.Softmax(dim=1) - softmax is not required in PyTorch as it is included in the CrossEntropyLoss
        )

    def forward(self, x):
        return self.model(x)



In [1]:
# set learning rate and epochs
epochs = 500

learning_rate = 0.05


# instantiate the model

model = MyNN(num_features=X_train.shape[1])

# define the loss function

loss_function = nn.CrossEntropyLoss()

# optimizer
optimizer = optim.SGD(model.parameters(), lr=learning_rate)


NameError: name 'MyNN' is not defined

In [ ]:
# training loop

for epoch in range(epochs):

    for batch_features, batch_labels in train_loader:

        # forward pass
        y_model_pred = model(batch_features)

        # loss calculation
        loss = loss_function(y_model_pred, batch_labels)

        # backprop

        optimizer.zero_grad_()
        loss.backward()

        optimizer.step()


        # update gradients